In [ ]:
# 04_CNN

In [ ]:
# 2. 간단한 모델 생성

# 모듈 import
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
import matplotlib.pyplot as plt

# mnist 손글씨 이미지 데이터 로드
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(f"train data size: {X_train.shape} {y_train.shape}")
print(f"test data size: {X_test.shape} {y_test.shape}")

# 샘플 이미지 출력
print("\ntrain 0번째 sample image:")
plt.figure(figsize=(2,2))
plt.imshow(X_train[0], cmap='gray')
plt.axis('off')
plt.show()

# train, test 의 최소, 최대값
print(f"X_train 최소값, 최대값: {X_train.min()}, {X_train.max()}")
print(f"X_test 최소값, 최대값: {X_test.min()}, {X_test.max()}\n")

# 정규화(Normalization)후 최대 최소값
X_train = X_train / 255.0
X_test = X_test / 255.0
print(f"정규화 후 X_train 최소값, 최대값: {X_train.min()}, {X_train.max()}")
print(f"정규화 후 X_test 최소값, 최대값: {X_test.min()}, {X_test.max()}\n")

# 채널 추가 (흑백, 컬러)
X_train_in = X_train[..., tf.newaxis]
X_test_in = X_test[:, :, :, None] # 위와 같음
print(f"채널 추가 전: {X_train.shape}, {X_test.shape}")
print(f"채널 추가 후: {X_train_in.shape}, {X_test_in.shape}\n")

# Functional API를 사용하여 샘플 모델 생성
inputs = Input(shape=(28, 28, 1), name='input')
x = Conv2D(32, 3, activation='relu', name='conv')(inputs)
x = MaxPooling2D(name='pool')(x)
x = Flatten(name='flatten')(x)
outputs = Dense(10, activation='softmax', name='output')(x)

model = Model(inputs=inputs, outputs=outputs)

# 모델 컴파일
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

# 모델 훈련
history = model.fit(
  X_train_in, y_train,
  validation_data=(X_test_in, y_test),
  epochs=10
)

evaluated = model.evaluate(X_test_in, y_test)
print(f"\n{evaluated}")

# 평가지표 시각화
def plot_loss_acc(history, epoch):
  loss, val_loss = history.history['loss'], history.history['val_loss']
  acc, val_acc = history.history['accuracy'], history.history['val_accuracy']
  
  fig, axes = plt.subplots(1, 2, figsize=(12, 4))
  
  axes[0].plot(range(1, epoch + 1), loss, label='Training')
  axes[0].plot(range(1, epoch + 1), val_loss, label='Validation')
  axes[0].legend(loc='best')
  axes[0].set_title("Loss")
  
  axes[1].plot(range(1, epoch + 1), acc, label='Training')
  axes[1].plot(range(1, epoch + 1), val_acc, label='Validation')
  axes[1].legend(loc='best')
  axes[1].set_title('Accuracy')
  
  plt.show()
plot_loss_acc(history, 10)

# 모델 구조
model.summary()

# 입력 텐서 형태
print("입력 텐서 형태")
print(model.inputs)

# 출력 텐서 형태
print("\n출력 텐서 형태")
print(model.outputs)

# 레이어 형태
print("\n레이어 형태")
print(model.layers)

# 첫번째 레이어 선택
print("\n첫번째 레이어 선택")
print(model.layers[0])

# 첫번째 레이어 입력
print("\n두번째 레이어 입력")
print(model.layers[1].input)

# 첫번째 레이어 출력
print("\n두번째 레이어 출력")
print(model.layers[1].output)

# 첫번째 레이어 가중치, 편향 (그 외의 추가적인 파라미터 입력으로 인한 가중치, 편향들)
print("\n두번째 레이어 가중치, 편향 (그 외의 추가적인 파라미터 입력으로 인한 가중치, 편향들)")
print(model.layers[1].weights)

# 첫번째 레이어 가중치만
print("\n두번째 레이어 가중치")
print(model.layers[1].kernel)

# 첫번째 레이어 편향만
print("\n두번째 레이어 편향")
print(model.layers[1].bias)

# 레이어 이름을 사용하여 레이어 선택
print("\n레이어 이름 사용하여 레이어 선택")
print(model.get_layer('conv'))

# 샘플 이미지의 레이어별 출력을 리스트에 추가
activator = Model(
  inputs=model.input,
  outputs=[layer.output for layer in model.layers[:3]]
)
activations = activator.predict(X_train_in[0][tf.newaxis, ...]) # 이미지 한장을 predict 하기 위해 배치 차원 추가
print(f"layer output 개수: {len(activations)}")

# 첫번째 레이어(conv) 출력층 크기
conv_activation = activations[1]
print(f"Conv2D layer shape: {conv_activation.shape}")

# Convolution 시각화
fig, axes = plt.subplots(4, 8)
fig.set_size_inches(10,5)

for i in range(32):
  ax = axes[i//8, i%8]
  ax.matshow(conv_activation[0, :, :, i], cmap='viridis')
  ax.set_title(f"kernel {i}", fontsize=10)
  plt.setp(ax.get_xticklabels(), visible=False)
  plt.setp(ax.get_yticklabels(), visible=False)
plt.tight_layout()
plt.show()

In [ ]:
# 3. 복잡한 모델 생성

import tensorflow as tf
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Concatenate
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

# mnist 손글씨 이미지 데이터 로드
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(f"train data size: {X_train.shape} {y_train.shape}")
print(f"test data size: {X_test.shape} {y_test.shape}\n")

# 새로운 학습 출력값 배열을 생성 (홀수: 1, 짝수: 0)
y_train_odd = []
for y in y_train:
  if y % 2 == 0:
    y_train_odd.append(0)
  else:
    y_train_odd.append(1)
y_train_odd = np.array(y_train_odd)
print(f"y_train_odd shape: {y_train_odd.shape}")
print(f"y_train 기본값: {y_train[:10]}")
print(f"y_train 홀수=1, 짝수=0: {y_train_odd[:10]}\n")

# 새로운 테스트 출력값 배열도 생성
y_test_odd = []
for y in y_test:
  if y % 2 == 0:
    y_test_odd.append(0)
  else:
    y_test_odd.append(1)
y_test_odd = np.array(y_test_odd)
print(f"y_test_odd shape: {y_test_odd.shape}")
print(f"y_test 기본값: {y_test[:10]}")
print(f"y_test 홀수=1, 짝수=0: {y_test_odd[:10]}\n")

# 정규화(Normalization)
X_train = X_train / 255.0
X_test = X_test / 255.0

# 채널 추가
# X_train_in = X_train[..., tf.newaxis]
X_train_in = tf.expand_dims(X_train, -1)
# X_test_in = x_test[..., tf.newaxis]
X_test_in = tf.expand_dims(X_test, -1)

print(f"채널 추가 전: {X_train.shape} {X_test_in.shape}")
print(f"채널 추가 후: {X_train_in.shape} {X_test_in.shape}")

# Functional API 를 사용하여 다중입력 모델 생성
inputs = Input(shape=(X_train_in.shape[1:]))

conv = Conv2D(32, 3, activation='relu')(inputs)
pool = MaxPooling2D()(conv)
flat = Flatten()(pool)

flat_inputs = Flatten()(inputs)

concat = Concatenate()([flat, flat_inputs])
outputs = Dense(10, activation='softmax')(concat)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

# 모델 구조 출력 및 이미지 파일로 저장
plot_model(
  model,
  to_file="functional_cnn.png",
  show_shapes=True,
  show_layer_names=True,
  dpi=100
)

# 모델 컴파일
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

# 모델 훈련
history = model.fit(
  X_train_in, y_train,
  epochs=10,
  validation_data=(X_test_in, y_test)
  # 원래 학습 검증용 데이터와 최종 평가에 사용되는 테스트 데이터는 중복 사용 하지 않는 것이 원칙
)

# 모델 성능
val_loss, val_acc = model.evaluate(X_test_in, y_test)
print(f"\n모델 성능: {val_loss}, {val_acc}")

#########################
#########################
# Functional API 를 사용하여 다중출력 모델 생성
inputs2 = Input(shape=(X_train_in.shape[1:]))

conv2 = Conv2D(32, 3, activation='relu', name='conv2d_layer')(inputs2)
pool2 = MaxPooling2D(name='maxpool_layer')(conv2)
flat2 = Flatten(name='flatten_layer')(pool2)

flat_inputs2 = Flatten()(inputs2)

concat2 = Concatenate()([flat2, flat_inputs2])
digit_outputs2 = Dense(10, activation='softmax', name='digit_dense')(concat2)

odd_outputs2 = Dense(1, activation='sigmoid', name='odd_dense')(flat_inputs2)

model2 = Model(
  inputs=inputs2,
  outputs=[digit_outputs2, odd_outputs2]
)

model2.summary()

# 모델의 입력과 출력을 나타내는 텐서
print(model2.input)
print(model2.output)

# 모델 구조 출력 및 이미지 파일로 저장
plot_model(
  model2,
  to_file="multi_output_cnn.png",
  show_shapes=True,
  show_layer_names=True,
  dpi=100
)

# 모델 컴파일
model2.compile(
  optimizer='adam',
  loss={
    'digit_dense': 'sparse_categorical_crossentropy',
    'odd_dense': 'binary_crossentropy'
  },
  loss_weights={
    'digit_dense': 1.0,
    'odd_dense': 0.5
  },
  metrics={
    'digit_dense': 'accuracy',
    'odd_dense': 'accuracy'
  }
)

# 입력값은 단순 입력, 나머지는 dic 타입으로 표기
history2 = model2.fit(
  X_train_in,
  {
    'digit_dense': y_train,
    'odd_dense': y_train_odd
  },
  validation_data=(
    X_test_in,
    {
      'digit_dense': y_test,
      'odd_dense': y_test_odd
    }
  ),
  epochs=10
)

# 모델 성능
evaluated2 = model2.evaluate(
  X_test_in,
  {
    'digit_dense': y_test,
    'odd_dense': y_test_odd
  }
)
print(f"\n{evaluated2}")

# 샘플 이미지 출력
def plot_image(data, i):
  plt.figure(figsize=(2,2))
  plt.imshow(data[i], cmap='gray')
  plt.axis('off')
  plt.show()
plot_image(X_test, 0)

# 예측값 출력
digit_preds, odd_preds = model2.predict(X_test_in)
print(f"\n클래스 확률 예측")
print(f"digit_preds[0]:\n{digit_preds[0]}")
print(f"odd_preds[0]:\n{odd_preds[0]}")

digit_labels = np.argmax(digit_preds, axis=-1)
print(f"\n클래스 확률 기반 라벨 예측")
print(digit_labels[:10])

odd_labels = (odd_preds > 0.5).astype(int).reshape(1, -1)[0]
print(f"\n0.5 기준으로 이진화된 예측값")
print(odd_labels[:10])

In [ ]:
print(model2.metrics_names,"\n")

for i in evaluated2:
  print(i)

print("\n")

for i, val in enumerate(evaluated2):
  print(f"{model2.metrics_names[i]}: {val}")